In [ ]:
import pandas as pd
import numpy as np
import json
import datetime as dt
from datetime import datetime, date, time,timedelta
import pandas as pd
from csv import DictWriter
import urllib
import re,datetime
import os
import time
from os import walk
import re
import  gc

############
#INPUTS
#############

index = "BANKNIFTY"
index_file_path = "D:/Dispersion/Dispersion Data - 2016_2020/Banknifty/BANKNIFTY.csv"

stfolder = "D:/Dispersion/Dispersion Data - 2016_2020/BN Dispersion"
    
wgt_lot_path = "D:/Dispersion/Dispersion Data - 2021/BankNifty Weights/BN Mcap & includeok - Copy.csv"

direc = r"C:/Users/admin/Desktop/Dispersion Working Files - 16.22 - Weekly Dispersion"

output_path = r"C:/Users/admin/Desktop/Dispersion output files check - 16.22 - Weekly Dispersion"

index_lotsize = 1

initial_equity = 100000

#########################
#Creating Stock List
#######################

filename = next(walk(stfolder), (None, None, []))[2]  # [] if no file
stock_list_path = []
for i in filename:
    temp = stfolder +"/"+ i
    stock_list_path.append(temp)

stock_list = []
for i in stock_list_path:
    stock_list.append(i.replace(".csv","").replace(stfolder + "/",""))
print(stock_list)
    
######################################    
#Creating weight and lot dictionary
######################################

"""wgtlot_df = pd.read_csv(wgt_lot_path)
wgtlot_dict = wgtlot_df.set_index('Symbol').T.to_dict('list')
print(wgtlot_dict)
symbols = list(wgtlot_dict.keys())"""

### Now adding Columns for weights below

####################################

lookback_period = 15
std_dev = 2

#################
#EXPIRY DATES
####################
symbols = stock_list.copy()
symbols.append(index)

exp_file_path_weekly = "expiry_dates_custom_week_numbers.csv"
exp_df_weekly = pd.read_csv(exp_file_path_weekly,parse_dates = ["Weekly_Expiry_Date","Date"],dayfirst =True,usecols = ["Weekly_Expiry_Date","Date"])

exp_file_path_monthly = "expiry_dates.csv"
exp_df_monthly = pd.read_csv(exp_file_path_monthly,parse_dates = ["curr_exp_date","Date"],dayfirst =True,usecols = ["curr_exp_date","Date"])
#exp_df.rename({'curr_date': 'Date'}, axis=1, inplace=True)

In [ ]:
idx = pd.read_csv(index_file_path,parse_dates = ["Date"],dayfirst = True)
df1 = idx[idx['Ticker'].str.endswith('- I')]
df2 = idx[idx['Ticker'].str.endswith('- II')]
df3 = idx[idx['Ticker'].str.endswith('- III')]

#no space between '-' & 'I' in weekly symbols

df4 = idx[idx['Ticker'].str.endswith('-I')]
df5 = idx[idx['Ticker'].str.endswith('-II')]
df6 = idx[idx['Ticker'].str.endswith('-III')]



index_file_path_monthly = direc + "/BANKNIFTY_MONTHLY.csv"

df1.to_csv(index_file_path_monthly)
df2.to_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_MONTHLY-II"))
df3.to_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_MONTHLY-III"))

df4.to_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_WEEKLY"))
df5.to_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_WEEKLY-II"))
df6.to_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_WEEKLY-III"))



print(idx.shape[0])
print(df1.shape[0]+df2.shape[0]+df3.shape[0], df4.shape[0]+df5.shape[0]+df6.shape[0])

    
for i in stock_list:
    df = pd.read_csv(stfolder + '/' + i + ".csv",parse_dates = ["Date"],dayfirst = True)
    df1 = df[df['Ticker'].str.endswith('- I')]
    df2 = df[df['Ticker'].str.endswith('- II')]
    df3 = df[df['Ticker'].str.endswith('- III')]
    
    path = direc + '/' + i + ".csv"
    
    df1.to_csv(path)
    df2.to_csv(path.replace(i,i+"-II"))
    df3.to_csv(path.replace(i,i+"-III"))
    
    print(df.shape[0])
    print(df1.shape[0]+df2.shape[0]+df3.shape[0])

stfolder = direc
print(index_file_path)

In [ ]:
####MONTHLY#####

idx = pd.read_csv(index_file_path_monthly,usecols = ["Date","Ticker","Option_Type","Strike",
                                                                    "OPT_Close","EQ_Close","IV","At The Money"],parse_dates = ["Date"])

idx = pd.merge(idx, exp_df_monthly, on = 'Date', how = 'left')


idx_ce = idx[(idx["Strike"] == idx["At The Money"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
idx_pe = idx[(idx["Strike"] == idx["At The Money"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")


final_monthly = idx_ce.merge(idx_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])

final_monthly = final_monthly.T.drop_duplicates().T  #Drop Duplicates Columns

final_monthly = final_monthly.add_prefix(index +"_Monthly_")
final_monthly = final_monthly.sort_values(by=index +"_Monthly_"+'Date_CE')

final_monthly = final_monthly.rename(columns = {'BANKNIFTY_Monthly_Date_CE':'BANKNIFTY_Date_CE'})

final_monthly.to_csv(output_path+"/check-1data_monthly.csv")


####WEEKLY#####

idx = pd.read_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_WEEKLY"),usecols = ["Date","Ticker","Option_Type","Strike",
                                                                    "OPT_Close","EQ_Close","IV","At The Money"],parse_dates = ["Date"])


idx = pd.merge(idx, exp_df_weekly, on = 'Date', how = 'left')

idx_ce = idx[(idx["Strike"] == idx["At The Money"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
idx_pe = idx[(idx["Strike"] == idx["At The Money"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")


final = idx_ce.merge(idx_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])

final = final.T.drop_duplicates().T  #Drop Duplicates Columns

final = final.add_prefix(index +"_Weekly_")
final = final.sort_values(by=index +"_Weekly_"+'Date_CE')

final = final.rename(columns = {'BANKNIFTY_Weekly_Date_CE':'BANKNIFTY_Date_CE'})


final = pd.merge(final_monthly, final,how='left',on=index + '_Date_CE')
display('final_weekly_merged', final)


final.to_csv(output_path+"/check-1data_weekly.csv")


####STOCKS#####

for i in stock_list:
    print(i)
    df1 = pd.read_csv(stfolder+ '/' + i + ".csv",parse_dates = ["Date"],usecols = ["Date","Ticker","Option_Type","Strike",
                                                                    "OPT_Close","EQ_Close","IV","At The Money"])


    df1 = pd.merge(df1, exp_df_monthly, on = 'Date', how = 'left')
    
    
    df_ce = df1[(df1["Strike"] == df1["At The Money"]) & (df1["Option_Type"] == "CE")].add_suffix("_CE")
    df_pe = df1[(df1["Strike"] == df1["At The Money"]) & (df1["Option_Type"] == "PE")].add_suffix("_PE")
    print(df1.shape[0], df_ce.shape[0], df_pe.shape[0])
    temp = df_ce.merge(df_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])
    temp = temp.T.drop_duplicates().T
    temp = temp.add_prefix(i +"_")
    
    final = final.merge(temp,how='left',left_on=index + '_Date_CE', right_on= i + '_Date_CE')
    
final = final.T.drop_duplicates().T
curr_final_df = final.copy()
u = curr_final_df.select_dtypes(exclude=['datetime'])
curr_final_df[u.columns] = u.fillna(0)

curr_final_df.to_csv(output_path+"/check-2data.csv")


In [ ]:
###MONTHLY#####

idx = pd.read_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_MONTHLY-II"),usecols = ["Date","Ticker","Option_Type","Strike",
                                                                    "OPT_Close","EQ_Close","IV","At The Money"],parse_dates = ["Date"])


idx = pd.merge(idx, exp_df_monthly, on = 'Date', how = 'left')


idx_ce = idx[(idx["Strike"] == idx["At The Money"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
idx_pe = idx[(idx["Strike"] == idx["At The Money"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")


final_monthly = idx_ce.merge(idx_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])

final_monthly = final_monthly.T.drop_duplicates().T  #Drop Duplicates Columns

final_monthly = final_monthly.add_prefix(index +"_Monthly_")
final_monthly = final_monthly.sort_values(by=index +"_Monthly_"+'Date_CE')
final_monthly = final_monthly.rename(columns = {'BANKNIFTY_Monthly_Date_CE':'BANKNIFTY_Date_CE'})

final_monthly.to_csv(output_path+"/check-3data_monthly.csv")

###WEEKLY#####

idx = pd.read_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_WEEKLY-II"),usecols = ["Date","Ticker","Option_Type","Strike",
                                                                    "OPT_Close","EQ_Close","IV","At The Money"],parse_dates = ["Date"])


idx = pd.merge(idx, exp_df_weekly, on = 'Date', how = 'left')


idx_ce = idx[(idx["Strike"] == idx["At The Money"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
idx_pe = idx[(idx["Strike"] == idx["At The Money"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")


final = idx_ce.merge(idx_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])

final = final.T.drop_duplicates().T  #Drop Duplicates Columns

final = final.add_prefix(index +"_Weekly_")
final = final.sort_values(by=index +"_Weekly_"+'Date_CE')
final = final.rename(columns = {'BANKNIFTY_Weekly_Date_CE':'BANKNIFTY_Date_CE'})


final = pd.merge(final_monthly, final,how='left',on=index + '_Date_CE')

final.to_csv(output_path+"/check-3data_weekly.csv")




for i in stock_list:
    print(i)
    path  = stfolder+ '/' + i + ".csv"
    df1 = pd.read_csv(path.replace(i,i+"-II"),parse_dates = ["Date"],usecols = ["Date","Ticker","Option_Type","Strike",
                                                                    "OPT_Close","EQ_Close","IV","At_The_Money_II"])
    df1 = pd.merge(df1, exp_df_monthly, on = 'Date', how = 'left')

    
    df_ce = df1[(df1["Strike"] == df1["At_The_Money_II"]) & (df1["Option_Type"] == "CE")].add_suffix("_CE")
    df_pe = df1[(df1["Strike"] == df1["At_The_Money_II"]) & (df1["Option_Type"] == "PE")].add_suffix("_PE")
    
    
    temp = df_ce.merge(df_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])
    temp = temp.T.drop_duplicates().T
    temp = temp.add_prefix(i +"_")
    final = final.merge(temp,how='left',left_on=index + '_Date_CE', right_on= i + '_Date_CE')
final = final.T.drop_duplicates().T
next_final_df = final.copy()

next_final_df.to_csv(output_path+"/check-4.0data.csv")


u = next_final_df.select_dtypes(exclude=['datetime'])
next_final_df[u.columns] = u.fillna(0)

next_final_df.to_csv(output_path+"/check-4data.csv")


In [ ]:
####MONTHLY####

replacedf = curr_final_df[curr_final_df[index + "_Date_CE"] == curr_final_df[index + "_Monthly_curr_exp_date_CE"]]
restdf = curr_final_df[~(curr_final_df[index + "_Date_CE"] == curr_final_df[index + "_Monthly_curr_exp_date_CE"])]

nextdf = next_final_df[next_final_df[index + "_Date_CE"] == next_final_df[index + "_Monthly_curr_exp_date_CE"]]


print(replacedf.shape[0])

print(restdf.shape[0])
print(nextdf.shape[0])
print(next_final_df.shape[0])


for i in symbols:
    if i == index:
        if nextdf.shape[0] == replacedf.shape[0]:
            replacedf[i+"_Monthly_IV_CE"] = nextdf[i+"_Monthly_IV_CE"]
            replacedf[i+"_Monthly_IV_PE"] = nextdf[i+"_Monthly_IV_PE"]
            
            print("EXCHANGE DONE")
        else:
            print("Entries Missing Error")
            break
    else:
        if nextdf.shape[0] == replacedf.shape[0]:
            replacedf[i+"_IV_CE"] = nextdf[i+"_IV_CE"]
            replacedf[i+"_IV_PE"] = nextdf[i+"_IV_PE"]
            print("EXCHANGE DONE")
        else:
            print("Entries Missing Error")
            break
        

final = restdf.append(replacedf).sort_values(by=[index+"_Date_CE"]).reset_index().drop(columns = ["index"])


####WEEKLY####

replacedfweekly = final[final[index + "_Date_CE"] == final[index + "_Weekly_Weekly_Expiry_Date_CE"]]
restdfweekly = final[~(final[index + "_Date_CE"] == final[index + "_Weekly_Weekly_Expiry_Date_CE"])]

nextdfweekly = next_final_df[next_final_df[index + "_Date_CE"] == next_final_df[index + "_Weekly_Weekly_Expiry_Date_CE"]]

for i in symbols:
    if i == index:
        if nextdf.shape[0] == replacedf.shape[0]:
            replacedfweekly[i+"_Weekly_IV_CE"] = nextdfweekly[i+"_Weekly_IV_CE"]
            replacedfweekly[i+"_Weekly_IV_PE"] = nextdfweekly[i+"_Weekly_IV_PE"]
            print("EXCHANGE DONE")
        else:
            print("Entries Missing Error")
            break
    else:
        continue
        
final = restdfweekly.append(replacedfweekly).sort_values(by=[index+"_Date_CE"]).reset_index().drop(columns = ["index"])


In [ ]:
final.to_csv(output_path+"\Part-1data.csv")
data = pd.read_csv(output_path+"\Part-1data.csv",parse_dates = [index + "_Date_CE"],dayfirst = True)

In [ ]:
#Adding Avg IV Columns
#Considering only monthly IV for index to calculate implied ratio

data[index + "_Monthly_Avg IV"] = (data[index + "_Monthly_IV_CE"] + data[index + "_Monthly_IV_PE"])/2
curr_final_df[index + "_Monthly_Avg IV"] = data[index + "_Monthly_Avg IV"]
for i in stock_list:
    data[i + "_Avg IV"] = (data[i + "_IV_CE"] + data[i + "_IV_PE"])/2
    curr_final_df[i + "_Avg IV"] = data[i + "_Avg IV"]
data = curr_final_df.copy()

# Adding Weights Columns

In [ ]:
weight = pd.read_csv(wgt_lot_path,parse_dates = ["date"],dayfirst = True,usecols = ["Security Symbol","Weightage","date","includeok","Lotsize"])

In [ ]:
grp = weight.groupby("Security Symbol")
data[index + "_OG_Weight"] = 100
for i in stock_list:
    d1 = grp.get_group(i)
    d1[i + "_OG_Weight"] = d1["Weightage"]
    d1[index+ "_Date_CE"] = d1["date"]
    data = pd.merge(data,d1[[index+ "_Date_CE",i + "_OG_Weight"]],on= index+ "_Date_CE", how='left')

if data.isnull().sum().sum() != 0:
    print(data.isnull().sum().sum())
    print("NULL VALUES PRESENT ERROR")
    print("NULL VALUES PRESENT ERROR")
    print("NULL VALUES PRESENT ERROR")

display(data)

### Adding Final Weight Columns

In [ ]:
filter_col = [col for col in data if col.endswith('Weight')]
filter_col.remove(index + "_OG_Weight")

data["actual_wgt_sum"] = data[filter_col].sum(axis=1)
for i in stock_list:
    data[i + "_Final_Weight"] = data[i + "_OG_Weight"]/data["actual_wgt_sum"]
data[index + "_Final_Weight"] = 1

In [ ]:
u = data.select_dtypes(exclude=['datetime'])
data[u.columns] = u.fillna(0)

display(data)

In [ ]:
data['index_iv'] = data[index +'_Monthly_Avg IV']*100
data['stock_iv']  = 0
for i in stock_list:
    data['stock_iv'] += data[i + '_Avg IV']*data[i + "_Final_Weight"] 
    
data['implied_correl'] = (data['index_iv']/data['stock_iv'])

print(data.isnull().sum().sum())

In [ ]:
data['avg'] = data['implied_correl'].rolling(lookback_period).mean()
df = data.copy()

df.to_csv(output_path+'/impliedratio.csv')

In [ ]:
def get_sma(prices, rate):
    return prices.rolling(rate).mean()

def get_bollinger_bands(prices, rate):
    sma = get_sma(prices, rate)
    std = prices.rolling(rate).std()
    bollinger_up = sma + std * std_dev # Calculate top band
    bollinger_down = sma - std * std_dev # Calculate bottom band
    return bollinger_up, bollinger_down

df['bollinger_up'], df['bollinger_down'] = get_bollinger_bands(df['implied_correl'], lookback_period)

df['sma'] = get_sma(df['implied_correl'], lookback_period)
df['long_avg'] = df['implied_correl'].mean()


#print(df['bollinger_up'])
#print(df['bollinger_down'])
#print(df['long_avg'])
#print(df['sma'])

In [ ]:
import matplotlib.pyplot as plt

df.head()

fig = plt.figure(figsize=(14,7))
plt.title(' Bollinger Bands')
plt.xlabel('Days')
plt.ylabel('Correl')
plt.plot(df['implied_correl'], label='Correl', c = 'black')
plt.plot(df['bollinger_up'], label='Bollinger Up', c='g')
plt.plot(df['bollinger_down'], label='Bollinger Down', c='r')
plt.plot(df['sma'], label='sma', c='b')

plt.legend()
plt.show()

In [ ]:
df.to_csv(output_path+"\series.csv")

In [ ]:
def implement_bb_strategy(data, lower_bb, upper_bb, avg):
    entry_ratio = []
    exit_ratio = []
    bb_signal = []
    signal = 0
    
    for i in range(len(data)):
        if(i==0):
            if data[i] < lower_bb[i]:
                if signal != 1:
                    entry_ratio.append(data[i])
                    exit_ratio.append(np.nan)
                    signal = 1
                    bb_signal.append(signal)
                else:
                    entry_ratio.append(np.nan)
                    exit_ratio.append(np.nan)
                    bb_signal.append(0)
            elif data[i] > upper_bb[i]:
                if signal != -1:
                    entry_ratio.append(np.nan)
                    exit_ratio.append(data[i])
                    signal = -1
                    bb_signal.append(signal)
                else:
                    entry_ratio.append(np.nan)
                    exit_ratio.append(np.nan)
                    bb_signal.append(0)
            else:
                entry_ratio.append(np.nan)
                exit_ratio.append(np.nan)
                bb_signal.append(0)
        else:
            if data[i-1] > lower_bb[i-1] and data[i] < lower_bb[i]:
                if signal != 1:
                    entry_ratio.append(data[i])
                    exit_ratio.append(np.nan)
                    signal = 1
                    bb_signal.append(signal)
                else:
                    entry_ratio.append(np.nan)
                    exit_ratio.append(np.nan)
                    bb_signal.append(bb_signal[i-1])

            elif data[i-1] > avg[i-1] and data[i] < avg[i]:
                if signal != 0:
                    entry_ratio.append(data[i])
                    exit_ratio.append(np.nan)
                    signal = 0
                    bb_signal.append(signal)
                else:
                    entry_ratio.append(np.nan)
                    exit_ratio.append(np.nan)
                    bb_signal.append(bb_signal[i-1])                
            
            elif data[i-1] < upper_bb[i-1] and data[i] > upper_bb[i]:
                if signal != -1:
                    entry_ratio.append(np.nan)
                    exit_ratio.append(data[i])
                    signal = -1
                    bb_signal.append(signal)
                else:
                    entry_ratio.append(np.nan)
                    exit_ratio.append(np.nan)
                    bb_signal.append(bb_signal[i-1])

            elif data[i-1] < avg[i-1] and data[i] > avg[i]:
                if signal != 0:
                    entry_ratio.append(np.nan)
                    exit_ratio.append(data[i])
                    signal = 0
                    bb_signal.append(signal)
                else:
                    entry_ratio.append(np.nan)
                    exit_ratio.append(np.nan)
                    bb_signal.append(bb_signal[i-1])

            else:
                entry_ratio.append(np.nan)
                exit_ratio.append(np.nan)
                bb_signal.append(bb_signal[i-1])

    return entry_ratio, exit_ratio, bb_signal

df['entry_ratio'], df['exit_ratio'], df['bb_signal'] = implement_bb_strategy(df['implied_correl'], df['bollinger_down'], df['bollinger_up'], df['avg'])

fig = plt.figure(figsize=(14,7))


df['implied_correl'].plot(label = 'Correlation', alpha = 1)
df['bollinger_up'].plot(label = 'UPPER BB', linestyle = '--', linewidth = 1, color = 'black')
df['avg'].plot(label = 'avg', linestyle = '--', linewidth = 1.2, color = 'grey')
df['bollinger_down'].plot(label = 'LOWER BB', linestyle = '--', linewidth = 1, color = 'black')
plt.scatter(df.index, df['entry_ratio'], marker = '^', color = 'green', label = 'BUY', s = 150)
plt.scatter(df.index, df['exit_ratio'], marker = 'v', color = 'red', label = 'SELL', s = 150)
plt.title('Bollinger Band')
plt.legend(loc = 'upper left')
plt.show()


In [ ]:
df1 = df.copy()

In [ ]:
df1['dispersion_entry'] = (df1['bb_signal'] == -1) & (df1['exit_ratio'] > 0)  & (df1['entry_ratio'].isna() == True)
df1['dispersion_exit'] = (df1['bb_signal'] == 0) & (df1['entry_ratio'] > 0) & (df1['entry_ratio'].isna() == False)

df1['correlation_entry'] = (df1['bb_signal'] == 1) & (df1['entry_ratio'] > 0)  & (df1['exit_ratio'].isna() == True)
df1['correlation_exit'] = (df1['bb_signal'] == 0)  & (df1['exit_ratio'] > 0)  & (df1['exit_ratio'].isna() == False)

In [ ]:
df2 = df1.copy()

df2.to_csv(output_path + '/ccccc.csv')

df2_index = df2.index
exp_dates_monthly = list(df2_index[df2[index + "_Monthly_curr_exp_date_CE"] == df2[index+"_Date_CE"]])
exp_dates_weekly = list(df2_index[df2[index + "_Weekly_Weekly_Expiry_Date_CE"] == df2[index+"_Date_CE"]])

print('exp_dates_monthly',exp_dates_monthly)
print('exp_dates_weekly',exp_dates_weekly)

for i in exp_dates_monthly:
    if df2.iloc[i]["bb_signal"] == -1:
        if df2.loc[i, 'dispersion_entry'] == True:
            continue
        else:
            df2.loc[i,'dispersion_entry'] = False
            df2.loc[i,'dispersion_exit'] = True
            df2.loc[i,'correlation_entry'] = False
            df2.loc[i,'correlation_exit'] = False
            df2.loc[i,'bb_signal'] = 0
            df2.loc[i,'entry_ratio'] = df2.iloc[i]["implied_correl"]
        try:
            if df2.loc[i+1, 'dispersion_exit'] == True:
                df2.loc[i+1,'dispersion_entry'] = False
                df2.loc[i+1,'dispersion_exit'] = False
                df2.loc[i+1,'correlation_entry'] = False
                df2.loc[i+1,'correlation_exit'] = False

            else:
                df2.loc[i+1,'dispersion_entry'] = True
                df2.loc[i+1,'dispersion_exit'] = False
                df2.loc[i+1,'correlation_entry'] = False
                df2.loc[i+1,'correlation_exit'] = False
                df2.loc[i+1,'bb_signal'] = -1
                df2.loc[i+1,'exit_ratio'] = df2.iloc[i+1]["implied_correl"]
        except Exception as e:
            print(e)
            continue
            
    elif df2.iloc[i]["bb_signal"] == 1:
        if df2.loc[i, 'correlation_entry'] == True:
            continue
        else:
            df2.loc[i,'dispersion_entry'] = False
            df2.loc[i,'dispersion_exit'] = False
            df2.loc[i,'correlation_entry'] = False
            df2.loc[i,'correlation_exit'] = True
            df2.loc[i,'bb_signal'] = 0
            df2.loc[i,'entry_ratio'] = df2.iloc[i]["implied_correl"]
        if df2.loc[i+1, 'correlation_exit'] == True:
            df2.loc[i+1,'dispersion_entry'] = False
            df2.loc[i+1,'dispersion_exit'] = False
            df2.loc[i+1,'correlation_entry'] = False
            df2.loc[i+1,'correlation_exit'] = False
        else:
            df2.loc[i+1,'dispersion_entry'] = False
            df2.loc[i+1,'dispersion_exit'] = False
            df2.loc[i+1,'correlation_entry'] = True
            df2.loc[i+1,'correlation_exit'] = False
            df2.loc[i+1,'bb_signal'] = 1
            df2.loc[i+1,'exit_ratio'] = df2.iloc[i+1]["implied_correl"]
        
    elif df2.iloc[i]["bb_signal"] == 0:
        continue


for i in exp_dates_weekly:        
    if df2.iloc[i]["bb_signal"] == -1:
        if df2.loc[i, 'dispersion_entry'] == True:
            continue
        else:
            df2.loc[i,'dispersion_entry'] = False
            df2.loc[i,'dispersion_exit'] = True
            df2.loc[i,'correlation_entry'] = False
            df2.loc[i,'correlation_exit'] = False
            df2.loc[i,'bb_signal'] = 0
            df2.loc[i,'entry_ratio'] = df2.iloc[i]["implied_correl"]
        try:

            if df2.loc[i+1, 'dispersion_exit'] == True:
                continue
            else:
                df2.loc[i+1,'dispersion_entry'] = True
                df2.loc[i+1,'dispersion_exit'] = False
                df2.loc[i+1,'correlation_entry'] = False
                df2.loc[i+1,'correlation_exit'] = False
                df2.loc[i+1,'bb_signal'] = -1
                df2.loc[i+1,'exit_ratio'] = df2.iloc[i+1]["implied_correl"]
        except Exception as e:
            print(e)
            continue
    elif df2.iloc[i]["bb_signal"] == 1:
        continue
    elif df2.iloc[i]["bb_signal"] == 0:
        continue

        
        

df = df2[ (df2["dispersion_entry"] == True) | (df2["correlation_exit"] == True) | 
              (df2["dispersion_exit"] == True) | (df2["correlation_entry"] == True) ].reset_index()

df2.to_csv(output_path + '/with entry exit signals_all data.csv')

In [ ]:
df = df2[ (df2["dispersion_entry"] == True) | (df2["correlation_exit"] == True) | 
              (df2["dispersion_exit"] == True) | (df2["correlation_entry"] == True) ].reset_index()


df.to_csv(output_path + '/with entry exit signals.csv')

In [ ]:
trades = []

for i in df.index[:-1]:
    if df["bb_signal"][i] == -1:
        strikes = {}
        strikes[index] = df[index + "_Weekly_Strike_CE"][i]
        for j in stock_list:
            strikes[j] = df[j+"_Strike_CE"][i]
        trades.append( 
                        { 
                          "entry_date": df[index + "_Date_CE"][i],
                          "exit_date" : df[index + "_Date_CE"][i+1],
                          "expiry_date" : df[index+"_Weekly_Weekly_Expiry_Date_CE"][i],
                          "type_of_trade": "SILS",
                          "strike" : strikes,
                          "entry_ratio" : df["exit_ratio"][i],
                          "exit_ratio" : df["entry_ratio"][i+1],
                            
                        }
                      )
                            
    if df["bb_signal"][i] == 1:
            strikes = {}
            strikes[index] = df[index + "_Monthly_Strike_CE"][i]
            for j in stock_list:
                strikes[j] = df[j+"_Strike_CE"][i]
            trades.append( 
                            { 
                              "entry_date": df[index + "_Date_CE"][i],
                              "exit_date" : df[index + "_Date_CE"][i+1],
                              "expiry_date" : df[index+"_Monthly_curr_exp_date_CE"][i],
                              "type_of_trade": "LISS",
                              "strike" : strikes,
                              "entry_ratio" : df["entry_ratio"][i],
                              "exit_ratio" : df["exit_ratio"][i+1],
                            }
                          )

In [ ]:
for i in trades:
    date1 = i["entry_date"]
    date2 = i["exit_date"]
    expiry = i["expiry_date"]
    trade = i["type_of_trade"]
    print(date1)
    print(date2)
    
    if expiry == date1:
        if trade == "SILS":
            df = pd.read_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_WEEKLY"),parse_dates = ["Date"])
            nextdf = pd.read_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_WEEKLY-II"),parse_dates = ["Date"])

            df = df[(df["Date"] > date1) & (df["Date"] <= date2) & (df["Strike"] == i["strike"][index])]
            nextdf = nextdf[(nextdf["Date"] == expiry) & (nextdf["Strike"] == i["strike"][index])]
            nextdf = nextdf.append(df)
            nextdf = pd.merge(nextdf, exp_df_weekly, on = 'Date', how = 'left')
            df_dict = {}
            df_dict[index] = nextdf
        elif trade == "LISS":
            df = pd.read_csv(index_file_path_monthly,parse_dates = ["Date"])
            nextdf = pd.read_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_MONTHLY-II"),parse_dates = ["Date"])

            df = df[(df["Date"] > date1) & (df["Date"] <= date2) & (df["Strike"] == i["strike"][index])]
            nextdf = nextdf[(nextdf["Date"] == expiry) & (nextdf["Strike"] == i["strike"][index])]
            nextdf = nextdf.append(df)
            nextdf = pd.merge(nextdf, exp_df_monthly, on = 'Date', how = 'left')
            df_dict = {}
            df_dict[index] = nextdf
        else:
            print("trade_type_error")
            break

        for j in stock_list:
            path = stfolder + '/' + j + ".csv"
            df = pd.read_csv(path,parse_dates = ["Date"])
            nextdf = pd.read_csv(path.replace(j,j+"-II"),parse_dates = ["Date"])
        
            df = df[(df["Date"] > date1) & (df["Date"] <= date2) & (df["Strike"] == i["strike"][j])]
            nextdf = nextdf[(nextdf["Date"] == expiry) & (nextdf["Strike"] == i["strike"][j])]
            nextdf = nextdf.append(df)
            nextdf = pd.merge(nextdf, exp_df_monthly, on = 'Date', how = 'left')
            df_dict[j] = nextdf
        i["dataframes"] = df_dict
    
    else:
        if trade == "SILS":
            df = pd.read_csv(index_file_path_monthly.replace("BANKNIFTY_MONTHLY","BANKNIFTY_WEEKLY"),parse_dates = ["Date"])
            df = df[(df["Date"] >= date1) & (df["Date"] <= date2) & (df["Strike"] == i["strike"][index])]
            df = pd.merge(df, exp_df_weekly, on = 'Date', how = 'left')
            df_dict = {}
            df_dict[index] = df

        elif trade == "LISS":
            df = pd.read_csv(index_file_path_monthly,parse_dates = ["Date"])
            df = df[(df["Date"] >= date1) & (df["Date"] <= date2) & (df["Strike"] == i["strike"][index])]
            df = pd.merge(df, exp_df_monthly, on = 'Date', how = 'left')
            df_dict = {}
            df_dict[index] = df
            
        else:
            print("trade_type_error")
            break

        for j in stock_list:
            df = pd.read_csv(stfolder + '/' + j + ".csv",parse_dates = ["Date"])
            df = df[(df["Date"] >= date1) & (df["Date"] <= date2) & (df["Strike"] == i["strike"][j])]
            df = pd.merge(df, exp_df_monthly, on = 'Date', how = 'left')
            df_dict[j] = df
        i["dataframes"] = df_dict

In [ ]:
for i in trades:
    print(i["dataframes"]['BANKNIFTY'].columns)

In [ ]:
result = pd.DataFrame()
for trade_dict in trades:
    df_dict = trade_dict["dataframes"]
    entry = trade_dict["entry_date"]
    exit = trade_dict["exit_date"]
    trade = trade_dict["type_of_trade"]
    
    if trade == "LISS":
        idx = df_dict[index][["Date","Ticker","OPT_Close","Strike","Option_Type","IV","EQ_Close","curr_exp_date"]]
    elif trade == "SILS":
        idx = df_dict[index][["Date","Ticker","OPT_Close","Strike","Option_Type","IV","EQ_Close","Weekly_Expiry_Date"]]
    else:
        print("trade_type_error")
        break
    
    idx_ce = idx[(idx["Option_Type"] == "CE")].add_suffix("_CE")
    idx_pe = idx[(idx["Option_Type"] == "PE")].add_suffix("_PE")

    final = idx_ce.merge(idx_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])
    final = final.add_prefix(index +"_")
    
    for i in df_dict:
        if i == index:
            continue
        df1 = df_dict[i][["Date","Ticker","OPT_Close","Strike","Option_Type","IV","EQ_Close","curr_exp_date"]]
        df_ce = df1[(df1["Option_Type"] == "CE")].add_suffix("_CE")
        df_pe = df1[(df1["Option_Type"] == "PE")].add_suffix("_PE")

        temp = df_ce.merge(df_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])
        temp = temp.T.drop_duplicates().T
        temp = temp.add_prefix(i +"_")
        
        final = final.merge(temp,left_on=index + '_Date_CE' , right_on= i + '_Date_CE', how = 'left')
    final["Trade_Type"] = trade_dict["type_of_trade"]
    
    conditions = [
    (final[index + '_Date_CE'] == entry),
    (final[index + '_Date_CE'] == exit),
    (final[index + '_Date_CE'] != exit) & (final[index + '_Date_CE'] != entry)]

    # create a list of the values we want to assign for each condition
    values = ['F', 'L', 'M']

    # create a new column and use np.select to assign values to it using our lists as arguments
    final['D_marker'] = np.select(conditions, values)
    result = result.append(final)
        
result = result.T.drop_duplicates().T

In [ ]:
data = result.copy()

In [ ]:
display(data)
data.to_csv(output_path + '/before_weight_add_second_time.csv')

### Adding Weight and Lot Columns Again

In [ ]:
weight = pd.read_csv(wgt_lot_path,parse_dates = ["date"],dayfirst = True,usecols = ["Security Symbol","Weightage","date","Lotsize"])

In [ ]:


grp = weight.groupby("Security Symbol")
data[index + "_OG_Weight"] = 100
data[index + "_Lotsize"] = index_lotsize
for i in stock_list:
    d1 = grp.get_group(i)
    d1[i + "_OG_Weight"] = d1["Weightage"]
    d1[i + "_Lotsize"] = d1["Lotsize"]
    d1[index+ "_Date_CE"] = d1["date"]
    data = pd.merge(data,d1[[index+ "_Date_CE",i + "_OG_Weight",i+ "_Lotsize"]],on= index+ "_Date_CE", how='left')

if data.isnull().sum().sum() != 0:
    print(data.isnull().sum().sum())
    print("NULL VALUES PRESENT ERROR")
    print("NULL VALUES PRESENT ERROR")
    print("NULL VALUES PRESENT ERROR")

u = data.select_dtypes(exclude=['datetime'])
data[u.columns] = u.fillna(0)

display(data)

data.to_csv(r'C:/Users/admin/Desktop/Dispersion output files check - 16.22/weigt_add_second_time.csv')

### Adding Final Weight Columns

In [ ]:
filter_col = [col for col in data if col.endswith('Weight')]
filter_col.remove(index + "_OG_Weight")

data["actual_wgt_sum"] = data[filter_col].sum(axis=1)
for i in stock_list:
    data[i + "_Final_Weight"] = data[i + "_OG_Weight"]/data["actual_wgt_sum"]
data[index + "_Final_Weight"] = 1

In [ ]:
filter_col = [col for col in data if col.endswith('Final_Weight')]
filter_col.remove(index + "_Final_Weight")
print(filter_col)

data[filter_col] = data[filter_col].replace(0,np.nan)

data["Min_Share_Value"] = data[filter_col].min(axis = 1)
data["Min_Share_Name"] = data[filter_col].idxmin(axis=1)
print(data["Min_Share_Value"])
print(data["Min_Share_Name"])

data[filter_col] = data[filter_col].replace(np.nan,0)

In [ ]:
display(data)
data.to_csv(output_path + '/before_qty_add.csv')

In [ ]:
#ADDING QTYS
result = data.copy()
min_exp = []
for idi, row in result.iterrows():
    min_share_stock = row['Min_Share_Name'].replace("_Final_Weight","")
    #print(min_share_stock)
    value  = row[min_share_stock + "_EQ_Close_CE"]*row[min_share_stock + "_Lotsize"]
    min_exp.append(value)
    
result["MinExp"] = min_exp
result[index + "_exp"] = (result["MinExp"]/result["Min_Share_Value"])
#result[index + "_lot"] = (result[index + "_exp"]/(result[index + "_Lotsize"]*result[index + "_EQ_Close_CE"])).astype(int)
result[index + "_lot"] = (result[index + "_exp"]/(result[index + "_Lotsize"]*result[index + "_EQ_Close_CE"]))
result[index + "_qty"] = result[index + "_lot"]*result[index + "_Lotsize"]

result.to_csv(output_path + '/resultt.csv')

#for stocks with includeok=0, making '_Lotsize' & '_EQ_Close_CE' non-zero

filter_col_Lotsize = [col for col in result if col.endswith('_Lotsize')]
filter_col_Lotsize.remove(index + "_Lotsize")
print(filter_col_Lotsize)
result[filter_col_Lotsize] = result[filter_col_Lotsize].replace(0,1)


filter_col_EQ_Close = [col for col in result if col.endswith('_EQ_Close_CE')]
filter_col_EQ_Close.remove(index + "_EQ_Close_CE")
print(filter_col_EQ_Close)
result[filter_col_EQ_Close] = result[filter_col_EQ_Close].replace(0,1)

result['index_exp'] = result[index + "_exp"]

result.to_csv(output_path + '/resulttcheckcheck.csv')

result['stock_exp'] = 0

for i in stock_list:
    print(i)
    if(i==min_share_stock):
        result[index + "_exp"]
        result[i + "_Final_Weight"]
        result[i + "_exp"] = result["MinExp"]
        result[i  + "_lot"] = 1
        result[i + "_qty"] = result[i + "_lot"]*result[i + "_Lotsize"]
        
        result['stock_exp'] += result[i + "_exp"]
        
    else:
        result[index + "_exp"]
        result[i + "_Final_Weight"]
        result[i + "_exp"] = (result[index + "_exp"]*result[i + "_Final_Weight"])
#         result[i  + "_lot"] = (result[i + "_exp"]/(result[i + "_Lotsize"]*result[i+ "_EQ_Close_CE"])).astype(int)
        result[i  + "_lot"] = (result[i + "_exp"]/(result[i + "_Lotsize"]*result[i+ "_EQ_Close_CE"]))
        result[i + "_qty"] = result[i + "_lot"]*result[i + "_Lotsize"]
        
        result['stock_exp'] += result[i + "_exp"]
        

In [ ]:
result['total_exposure'] = (result['index_exp'] + result['stock_exp'])*2

conditions = [
    (result["Trade_Type"] == "LISS"),
    (result["Trade_Type"] == "SILS")]

values = [result['total_exposure']/10,result['total_exposure']/10]
result['margin'] = np.select(conditions, values)

result['units'] = (initial_equity/result['margin']).astype(int)

result.to_csv(output_path+"/final2.csv")

In [ ]:
conditions = [
    (result["D_marker"] == "F"),
    (result["D_marker"] != "F")]

values = [result[index + "_qty"]*result['units'],np.nan]
result[index + '_qty_final'] = np.select(conditions, values)

for i in stock_list:
    values = [result[i + "_qty"]*result['units'],np.nan]
    result[i + '_qty_final'] = np.select(conditions, values)

In [ ]:
result = result.ffill()

In [ ]:
display(result)

# Adding Next mOnth Close COlumns 

In [ ]:
# for i in symbols:
#     for j in ["PE","CE"]:
#         temp = []
#         for idi, row in result.iterrows():
#             strike = row[i+"_Strike_CE"]
#             date = row[index+"_Date_CE"]
#             path = direc + "/" + i + "-II.csv" 

#             df1 = pd.read_csv(path,parse_dates = ["Date"],usecols = ["Date","Ticker","Option_Type","Strike",
#                                                                             "OPT_Close","EQ_Close","IV","At_The_Money_II"])
#             df1 = df1[(df1["Date"] == date)&(df1["Strike"]==strike)&(df1["Option_Type"]==j)]
#             if df1.shape[0] == 1:
#                 print("1 entry only!")
#             else:
#                 print(i)
#                 print(df1)
#                 print(date)
#                 print(strike)
#                 print(j)
#                 print("Error")  
#                 break
#             close2 = list(df1["OPT_Close"])[0]
#             temp.append(close2)
#         result[i + "_Next_Month_Close_"+j] = temp
# print(print(result.isnull().sum().sum()))
# display(result)

In [ ]:
# print(symbols)
# print(result.isnull().sum().sum())
# resultp1 = result[(result[index+ "_Date_CE"] == result[index+ "_curr_exp_date_CE"]) & (result["D_marker"] == "M")]
# resultp2 = result[~((result[index+ "_Date_CE"] == result[index+ "_curr_exp_date_CE"]) & (result["D_marker"] == "M"))]

# if resultp1.shape[0] + resultp2.shape[0] == result.shape[0]:
#     print("Check Successful")
# else:
#     raise Exception("Check Unsuccessful")

# for i in symbols:
#     resultp2[i+"_Expiry_PL_Adj_CE"] = 0
#     resultp2[i+"_Expiry_PL_Adj_PE"] = 0

# if resultp1.shape[0] == 0:
#     print("No_expiry_adj")
# else:
#     for i in symbols:
#         for j in ["CE","PE"]:
#             temp = []
#             for idi, row in resultp1.iterrows():
#                 strike = row[i+"_Strike_CE"]
#                 date = row[index+"_Date_CE"]
#                 close = row[i + "_OPT_Close_"+ j]
#                 tradetype = row["Trade_Type"]
                
#                 path = direc + "/" + i + "-II.csv" 
                
#                 df1 = pd.read_csv(path,parse_dates = ["Date"],usecols = ["Date","Ticker","Option_Type","Strike",
#                                                                                 "OPT_Close","EQ_Close","IV","At The Money"])
#                 df1 = df1[(df1["Date"] == date)&(df1["Strike"]==strike)&(df1["Option_Type"]==j)]
                
#                 if df1.shape[0] == 1:
#                     print("1 entry only!")
#                 else:
#                     print("Error")  
#                     break
    
#                 close2 = list(df1["OPT_Close"])[0]
                
#                 if tradetype == "SILS":
#                     if i == index:
#                         adj = close2 - close
#                     else:
#                         adj = close - close2
#                 elif tradetype == "LISS":
#                     if i == index:
#                         adj = close - close2
#                     else:
#                         adj = close2 - close
#                 else:
#                     print("Error")
#                     break
#                 temp.append(adj)
#             resultp1[i+ "_Expiry_PL_Adj_"+ j] = temp
# print(resultp1.isnull().sum().sum())
# print(resultp2.isnull().sum().sum())
# result = resultp1.append(resultp2).sort_values(by=[index+"_Date_CE"]).reset_index().drop(columns = ["index"])

In [ ]:
####FOR NEXT MONTH CLOSE COLUMN

# plvalue = []

# for idi, row in result.iterrows():
#     pval = 0
#     if row["D_marker"] == "F":
#         pl = 0
        
#     else:
#         if result.loc[idi-1,"BANKNIFTY_curr_exp_date_CE"] == result.loc[idi-1,"BANKNIFTY_Date_CE"]:
#             if row["Trade_Type"] == "SILS":
#                 plvalce = row[index + "_qty_final"]*(result.loc[idi-1, index + "_Next_Month_Close_CE"] - result.loc[idi, index + "_OPT_Close_CE"])
#                 plvalpe = row[index + "_qty_final"]*(result.loc[idi-1, index + "_Next_Month_Close_PE"] - result.loc[idi, index + "_OPT_Close_PE"])
#                 pval = plvalce + plvalpe
                
#                 for i in stock_list:
#                     plvalce = row[i + "_qty_final"]*(result.loc[idi, i + "_OPT_Close_CE"] - result.loc[idi-1, i + "_Next_Month_Close_CE"])
#                     plvalpe = row[i + "_qty_final"]*(result.loc[idi, i + "_OPT_Close_PE"] - result.loc[idi-1, i + "_Next_Month_Close_PE"])
#                     pval += plvalce + plvalpe
                    
#             if row["Trade_Type"] == "LISS":
#                 plvalce = row[index + "_qty_final"]*(result.loc[idi, index + "_OPT_Close_CE"] - result.loc[idi-1, index + "_Next_Month_Close_CE"])
#                 plvalpe = row[index + "_qty_final"]*(result.loc[idi, index + "_OPT_Close_PE"] - result.loc[idi-1, index + "_Next_Month_Close_PE"])
#                 pval = plvalce + plvalpe
                
#                 for i in stock_list:
#                     plvalce = row[i + "_qty_final"]*(result.loc[idi-1, i + "_Next_Month_Close_CE"] - result.loc[idi, i + "_OPT_Close_CE"])
#                     plvalpe = row[i + "_qty_final"]*(result.loc[idi-1, i + "_Next_Month_Close_PE"] - result.loc[idi, i + "_OPT_Close_PE"])
#                     pval += plvalce + plvalpe
                    
#         else:
#             if row["Trade_Type"] == "SILS":
#                 plvalce = row[index + "_qty_final"]*(result.loc[idi-1, index + "_OPT_Close_CE"] - result.loc[idi, index + "_OPT_Close_CE"])
#                 plvalpe = row[index + "_qty_final"]*(result.loc[idi-1, index + "_OPT_Close_PE"] - result.loc[idi, index + "_OPT_Close_PE"])
#                 pval = plvalce + plvalpe
                
#                 for i in stock_list:
#                     plvalce = row[i + "_qty_final"]*(result.loc[idi, i + "_OPT_Close_CE"] - result.loc[idi-1, i + "_OPT_Close_CE"])
#                     plvalpe = row[i + "_qty_final"]*(result.loc[idi, i + "_OPT_Close_PE"] - result.loc[idi-1, i + "_OPT_Close_PE"])
#                     pval += plvalce + plvalpe
                    
#             if row["Trade_Type"] == "LISS":
#                 plvalce = row[index + "_qty_final"]*(result.loc[idi, index + "_OPT_Close_CE"] - result.loc[idi-1, index + "_OPT_Close_CE"])
#                 plvalpe = row[index + "_qty_final"]*(result.loc[idi, index + "_OPT_Close_PE"] - result.loc[idi-1, index + "_OPT_Close_PE"])
#                 pval = plvalce + plvalpe
                
#                 for i in stock_list:
#                     plvalce = row[i + "_qty_final"]*(result.loc[idi-1, i + "_OPT_Close_CE"] - result.loc[idi, i + "_OPT_Close_CE"])
#                     plvalpe = row[i + "_qty_final"]*(result.loc[idi-1, i + "_OPT_Close_PE"] - result.loc[idi, i + "_OPT_Close_PE"])
#                     pval += plvalce + plvalpe
                    
        
        
        
        
#     plvalue.append(pval)

# result["Daily_PL"] = plvalue

# result.to_csv(output_path+"\pl_final4.csv")


In [ ]:

plvalue = []

pl = 0

for idi, row in result.iterrows():
    pval = 0
    if row["D_marker"] == "F":

        result.loc[idi, 'pl_check'] = pl + initial_equity
        result.loc[idi+1:, 'pl_check'] = np.nan

        result.loc[idi , 'units_new'] = ((result.loc[idi ,'pl_check'])/result.loc[idi, 'margin'])
        result.loc[idi+1: , 'units_new'] = np.nan
       
        result.loc[idi, index + '_qty_final'] = result.loc[idi, index + "_qty"] * result.loc[idi, 'units_new']
        result.loc[idi+1:, index + '_qty_final'] = np.nan

        for i in stock_list:                                                                    
            result.loc[idi, i + '_qty_final'] = result.loc[idi, i + "_qty"] * result.loc[idi, 'units_new']
            result.loc[idi+1:, i + '_qty_final'] = np.nan
        
        result = result.ffill()
           
    else:
        if row["Trade_Type"] == "SILS":
            plvalce = result.loc[idi, index + "_qty_final"]*(result.loc[idi-1, index + "_OPT_Close_CE"] - result.loc[idi, index + "_OPT_Close_CE"])
            plvalpe = result.loc[idi, index + "_qty_final"]*(result.loc[idi-1, index + "_OPT_Close_PE"] - result.loc[idi, index + "_OPT_Close_PE"])
            pval = plvalce + plvalpe

            for i in stock_list:
                plvalce = result.loc[idi, i + "_qty_final"]*(result.loc[idi, i + "_OPT_Close_CE"] - result.loc[idi-1, i + "_OPT_Close_CE"])
                plvalpe = result.loc[idi, i + "_qty_final"]*(result.loc[idi, i + "_OPT_Close_PE"] - result.loc[idi-1, i + "_OPT_Close_PE"])
                pval += plvalce + plvalpe

        if row["Trade_Type"] == "LISS":
            plvalce = result.loc[idi, index + "_qty_final"]*(result.loc[idi, index + "_OPT_Close_CE"] - result.loc[idi-1, index + "_OPT_Close_CE"])
            plvalpe = result.loc[idi, index + "_qty_final"]*(result.loc[idi, index + "_OPT_Close_PE"] - result.loc[idi-1, index + "_OPT_Close_PE"])
            pval = plvalce + plvalpe

            for i in stock_list:
                plvalce = result.loc[idi, i + "_qty_final"]*(result.loc[idi-1, i + "_OPT_Close_CE"] - result.loc[idi, i + "_OPT_Close_CE"])
                plvalpe = result.loc[idi, i + "_qty_final"]*(result.loc[idi-1, i + "_OPT_Close_PE"] - result.loc[idi, i + "_OPT_Close_PE"])
                pval += plvalce + plvalpe

        
        pl += pval
        
        
    plvalue.append(pval)

result["Daily_PL"] = plvalue

result.to_csv(output_path+"\pl_final4.csv")


In [ ]:
result.loc[0,'Daily_PL'] = initial_equity
result["PortfolioValue"] = result["Daily_PL"].cumsum()
result.loc[0,'Daily_PL'] = 0

display(result)

In [ ]:
result.to_csv(output_path+"/final4.1.csv")

fig = plt.figure(figsize=(14,7))
plt.title('Equity Curve')
plt.xlabel('Days')
plt.ylabel('PortfolioValue')
plt.plot((result["BANKNIFTY_Date_CE"]).astype('str'),result['PortfolioValue'], c = 'blue')

plt.xticks(rotation=45)

plt.legend()
plt.show()

In [ ]:
finallist = pd.DataFrame()
tradelist = result[ (result["D_marker"] == "F") | (result["D_marker"] == "L") ]

if list(tradelist.iloc[[-1]]["D_marker"])[0] == "F":
    tradelist.drop(tradelist.tail(1).index,inplace=True)
else:
    tradelist = tradelist

tr1 = tradelist[tradelist["D_marker"]=="F"].reset_index().drop(columns = ["index"])
tr2 = tradelist[tradelist["D_marker"]=="L"].reset_index().drop(columns = ["index"])

print(tr1.shape[0])
if tr1.shape[0] == tr2.shape[0]:
    for idx in tr1.index:
        mdict = {}
        mdict["Entry_Date"] = tr1[index+ "_Date_CE"][idx]
        mdict["Exit_Date"] = tr2[index+ "_Date_CE"][idx]
        mdict["Trade_Type"] = tr1["Trade_Type"][idx]
        for i in symbols:
            for j in ["CE","PE"]:
                mdict[i + "_Entry_Price_" + j] = tr1[i + "_OPT_Close_" + j][idx] 
                mdict[i + "_Exit_Price_" + j] = tr2[i + "_OPT_Close_" + j][idx]
                mdict[i + "_Qty_" + j] = tr1[i + "_qty_final"][idx]
        temp = pd.DataFrame([mdict])
        #display(temp)
        finallist = finallist.append(temp)
else:
    print("Error, Tradelist not proper")

finallist.reset_index().drop(columns = ["index"])

# Adding Expiry Roll OVer Entry Exit

In [ ]:
# finallist_exp = pd.DataFrame()
# expirytrades = result[(result[index+ "_Date_CE"] == result[index+ "_curr_exp_date_CE"]) & (result["D_marker"] == "M")]
# #display(expirytrades)
# for idi, row1 in finallist.iterrows():
#     flag = 0
#     for idi, row2 in expirytrades.iterrows():
#         expdate = row2[index+ "_Date_CE"]
#         d1 = row1["Entry_Date"]
#         d2 = row1["Exit_Date"]
#         #print(expdate)
#         #print(d1)
#         #print(d2)
#         if (d1 < expdate < d2) & (flag ==  0):
#             mdict = {}
#             mdict["Expiry_Date"] = expdate
#             for i in symbols:
#                 for j in ["CE","PE"]:
#                     mdict[i + "_Exp_Adj_Entry_Price_" + j] = row2[i + "_Next_Month_Close_" + j]
#                     mdict[i + "_Exp_Adj_Exit_Price_" + j] = row2[i + "_OPT_Close_" + j]
#             temp = pd.DataFrame([mdict])
#             #display(temp)
#             finallist_exp = finallist_exp.append(temp)
#             flag = 1
#     if flag == 0:
#         mdict = {}
#         mdict["Expiry_Date"] = "NA"
#         for i in symbols:
#             for j in ["CE","PE"]:
#                 mdict[i + "_Exp_Adj_Entry_Price_" + j] = 0
#                 mdict[i + "_Exp_Adj_Exit_Price_" + j] = 0
#         temp = pd.DataFrame([mdict])
#         #display(temp)
#         finallist_exp = finallist_exp.append(temp)


# finallist_exp.reset_index().drop(columns = ["index"])

In [ ]:
#tradelist =  pd.concat([finallist, finallist_exp], axis=1)

tradelist = finallist

In [ ]:
display(tradelist)

In [ ]:
tradelist.to_csv(output_path+"/trdlist.csv")

In [ ]:
#####FOR NEXT MONTH ROLLOVER############

# pl_tradelist = tradelist.copy()
# plvalue = []
# pval = 0
# for idi, row in pl_tradelist.iterrows():

#     if row["Trade_Type"] == "SILS":
#         pvalce = row[index + "_Qty_CE"]*((row[index + "_Entry_Price_CE"] - row[index + "_Exp_Adj_Exit_Price_CE"])+(row[index + "_Exp_Adj_Entry_Price_CE"] - row[index + "_Exit_Price_CE"]))
#         pvalpe = row[index + "_Qty_PE"]*((row[index + "_Entry_Price_PE"] - row[index + "_Exp_Adj_Exit_Price_PE"])+(row[index + "_Exp_Adj_Entry_Price_PE"] - row[index + "_Exit_Price_PE"]))
#         pval = pvalce + pvalpe

#         for i in stock_list:
#             pvalce = row[i + "_Qty_CE"]*((row[i + "_Exp_Adj_Exit_Price_CE"] - row[i + "_Entry_Price_CE"])+(row[i + "_Exit_Price_CE"] - row[i + "_Exp_Adj_Entry_Price_CE"]))
#             pvalpe = row[i + "_Qty_PE"]*((row[i + "_Exp_Adj_Exit_Price_PE"] - row[i + "_Entry_Price_PE"])+(row[i + "_Exit_Price_PE"] - row[i + "_Exp_Adj_Entry_Price_PE"]))
#             pval += pvalce + pvalpe    

#     if row["Trade_Type"] == "LISS":
#         pvalce = row[index + "_Qty_CE"]*((row[index + "_Exp_Adj_Exit_Price_CE"] - row[index + "_Entry_Price_CE"])+(row[index + "_Exit_Price_CE"] - row[index + "_Exp_Adj_Entry_Price_CE"]))
#         pvalpe = row[index + "_Qty_PE"]*((row[index + "_Exp_Adj_Exit_Price_PE"] - row[index + "_Entry_Price_PE"])+(row[index + "_Exit_Price_PE"] - row[index + "_Exp_Adj_Entry_Price_PE"]))
#         pval = pvalce + pvalpe

#         for i in stock_list:
#             pvalce = row[i + "_Qty_CE"]*((row[i + "_Entry_Price_CE"] - row[i + "_Exp_Adj_Exit_Price_CE"])+(row[i + "_Exp_Adj_Entry_Price_CE"] - row[i + "_Exit_Price_CE"]))
#             pvalpe = row[i + "_Qty_PE"]*((row[i + "_Entry_Price_PE"] - row[i + "_Exp_Adj_Exit_Price_PE"])+(row[i + "_Exp_Adj_Entry_Price_PE"] - row[i + "_Exit_Price_PE"]))
#             pval += pvalce + pvalpe    

#     plvalue.append(pval)
                    
# pl_tradelist['pl'] = plvalue
# pl_tradelist.to_csv(output_path+"\pl_trdlist.csv")




In [ ]:
pl_tradelist = tradelist.copy()
plvalue = []
pval = 0
for idi, row in pl_tradelist.iterrows():

    if row["Trade_Type"] == "SILS":
        pvalce = row[index + "_Qty_CE"]*(row[index + "_Entry_Price_CE"] - row[index + "_Exit_Price_CE"])
        pvalpe = row[index + "_Qty_PE"]*(row[index + "_Entry_Price_PE"] - row[index + "_Exit_Price_PE"])
        pval = pvalce + pvalpe

        for i in stock_list:
            pvalce = row[i + "_Qty_CE"]*(row[i + "_Exit_Price_CE"] - row[i + "_Entry_Price_CE"])
            pvalpe = row[i + "_Qty_PE"]*(row[i + "_Exit_Price_PE"] - row[i + "_Entry_Price_PE"])
            pval += pvalce + pvalpe    

    if row["Trade_Type"] == "LISS":
        pvalce = row[index + "_Qty_CE"]*(row[index + "_Exit_Price_CE"] - row[index + "_Entry_Price_CE"])
        pvalpe = row[index + "_Qty_PE"]*(row[index + "_Exit_Price_PE"] - row[index + "_Entry_Price_PE"])
        pval = pvalce + pvalpe

        for i in stock_list:
            pvalce = row[i + "_Qty_CE"]*(row[i + "_Entry_Price_CE"] - row[i + "_Exit_Price_CE"])
            pvalpe = row[i + "_Qty_PE"]*(row[i + "_Entry_Price_PE"] - row[i + "_Exit_Price_PE"])
            pval += pvalce + pvalpe    

    plvalue.append(pval)
                    
pl_tradelist['pl'] = plvalue
pl_tradelist.to_csv(output_path+"\pl_trdlist.csv")




### Calculate CAGR 

In [4]:
firstValue = round(df.loc[0, 'PortfolioValue'],4)
lastValue = round(df.loc[df.index[-1], 'PortfolioValue'],4)
firstValue, lastValue

(100000.0, 405153.5174)

In [5]:
cagr = ((lastValue/firstValue) ** (1/5) - 1 ) * 100
cagr

32.28905762688419

### Calculate Daily Drawdown

In [6]:
Roll_Max = round(df['PortfolioValue'].expanding().max(), 2)
Daily_Drawdown = Roll_Max/round(df['PortfolioValue'], 2) - 1.0


In [7]:
#Roll_Max[0:50], Daily_Drawdown[0:50]

(0     100000.00
 1     100000.00
 2     100000.00
 3     100000.00
 4     100000.00
 5     100000.00
 6     100000.00
 7     100000.00
 8     100000.00
 9     100000.00
 10    100000.00
 11    100000.00
 12    100000.00
 13    100000.00
 14    100000.00
 15    100000.00
 16    100000.00
 17    100000.00
 18    101831.15
 19    101831.15
 20    101831.15
 21    101831.15
 22    101831.15
 23    101831.15
 24    101831.15
 25    101831.15
 26    101831.15
 27    102559.97
 28    105246.93
 29    105692.75
 30    105692.75
 31    105692.75
 32    105692.75
 33    105692.75
 34    105692.75
 35    105692.75
 36    105692.75
 37    107148.56
 38    109434.36
 39    109434.36
 40    110203.58
 41    113367.36
 42    117106.20
 43    117106.20
 44    117106.20
 45    117106.20
 46    118146.67
 47    118146.67
 48    118172.13
 49    119188.01
 Name: PortfolioValue, dtype: float64,
 0     0.000000
 1     0.001992
 2     0.002495
 3     0.004871
 4     0.008731
 5     0.010708
 6     0.010791

In [8]:
df['Daily_Drawdown'] = Daily_Drawdown * 100


In [9]:
df.to_csv('DailyDrawdown.csv', index=False)

### Monthly PNL Percentage 

In [10]:
df['BANKNIFTY_Date_CE'] = pd.to_datetime(df['BANKNIFTY_Date_CE'], dayfirst=True)

In [11]:
df['Year'] = pd.DatetimeIndex(df['BANKNIFTY_Date_CE']).year
df['Month'] = pd.DatetimeIndex(df['BANKNIFTY_Date_CE']).month
df.to_csv('df.csv', index=False)

In [12]:
i = 0
dfg = df.groupby(['Year', 'Month'])
for name, group in dfg:
    #print(name)
    if i == 0: 
        firstValue = group['PortfolioValue'].iloc[0]
        i = 1
    else:
        firstValue = lastValue

    lastValue = group['PortfolioValue'].loc[group.index[-1]]
    change = lastValue/firstValue - 1
    df.loc[group.index, 'Change_%_Monthly'] = round(change * 100, 2)
    
i = 0
dfg = df.groupby(['Year'])
for name, group in dfg:
    #print(name)
    if i == 0: 
        firstValue = group['PortfolioValue'].iloc[0]
        i = 1
    else:
        firstValue = lastValue

    lastValue = group['PortfolioValue'].loc[group.index[-1]]
    change = lastValue/firstValue - 1
    df.loc[group.index, 'Change_%_Yearly'] = change * 100
    
df

,Unnamed: 0,BANKNIFTY_Date_CE,BANKNIFTY_Ticker_CE,BANKNIFTY_OPT_Close_CE,BANKNIFTY_Strike_CE,BANKNIFTY_IV_CE,BANKNIFTY_EQ_Close_CE,BANKNIFTY_curr_exp_date_CE,BANKNIFTY_Ticker_PE,BANKNIFTY_OPT_Close_PE,BANKNIFTY_IV_PE,AXISBANK_Ticker_CE,AXISBANK_OPT_Close_CE,AXISBANK_Strike_CE,AXISBANK_IV_CE,AXISBANK_EQ_Close_CE,AXISBANK_Ticker_PE,AXISBANK_OPT_Close_PE,AXISBANK_IV_PE,BANDHANBNK_Date_CE,BANDHANBNK_Ticker_CE,BANDHANBNK_OPT_Close_CE,BANDHANBNK_Strike_CE,BANDHANBNK_IV_CE,BANDHANBNK_EQ_Close_CE,BANDHANBNK_curr_exp_date_CE,BANDHANBNK_Date_PE,BANDHANBNK_Ticker_PE,BANDHANBNK_OPT_Close_PE,BANDHANBNK_IV_PE,BANKBARODA_Ticker_CE,BANKBARODA_OPT_Close_CE,BANKBARODA_Strike_CE,BANKBARODA_IV_CE,BANKBARODA_EQ_Close_CE,BANKBARODA_Ticker_PE,BANKBARODA_OPT_Close_PE,BANKBARODA_IV_PE,BANKINDIA_Date_CE,BANKINDIA_Ticker_CE,BANKINDIA_OPT_Close_CE,BANKINDIA_Strike_CE,BANKINDIA_IV_CE,BANKINDIA_EQ_Close_CE,BANKINDIA_curr_exp_date_CE,BANKINDIA_Ticker_PE,BANKINDIA_OPT_Close_PE,BANKINDIA_IV_PE,CANBK_Date_CE,CANBK_Ticker_CE,...,ICICIBANK_qty,IDFCFIRSTB_exp,IDFCFIRSTB_lot,IDFCFIRSTB_qty,INDUSINDBK_exp,INDUSINDBK_lot,INDUSINDBK_qty,KOTAKBANK_exp,KOTAKBANK_lot,KOTAKBANK_qty,PNB_exp,PNB_lot,PNB_qty,RBLBANK_exp,RBLBANK_lot,RBLBANK_qty,SBIN_exp,SBIN_lot,SBIN_qty,YESBANK_exp,YESBANK_lot,YESBANK_qty,total_exposure,margin,units,BANKNIFTY_qty_final,AXISBANK_qty_final,BANDHANBNK_qty_final,BANKBARODA_qty_final,BANKINDIA_qty_final,CANBK_qty_final,FEDERALBNK_qty_final,HDFCBANK_qty_final,ICICIBANK_qty_final,IDFCFIRSTB_qty_final,INDUSINDBK_qty_final,KOTAKBANK_qty_final,PNB_qty_final,RBLBANK_qty_final,SBIN_qty_final,YESBANK_qty_final,pl_check,units_new,Daily_PL,PortfolioValue,Daily_Drawdown,Year,Month,Change_%_Monthly,Change_%_Yearly
0,0,2016-01-01,BANKNIFTY17000CE - I,369.0500,17000.0,0.1730,17033.8008,2016-01-28,BANKNIFTY17000PE - I,272.50,0.1730,AXISBANK450CE - I,14.70,450.0,0.2858,449.90,AXISBANK450PE - I,12.10,0.2643,NaN,0,0.00,0.0,0.0000,1.00,NaN,0,0,0.00,0.0000,BANKBARODA160CE - I,5.80,160.0,0.3465,159.05,BANKBARODA160PE - I,6.10,0.3431,2016-01-01,BANKINDIA117.5CE - I,4.20,117.5,0.3194,117.35,2016-01-28,BANKINDIA117.5PE - I,3.05,0.2511,2016-01-01,CANBK240CE - I,...,7.315681,0.000000,0.000000,0.000000,719.574725,0.746563,0.746563,1677.718132,2.306935,2.306935,290.151099,2.467271,2.467271,0.000000,0.000000,0.000000,2225.781319,9.770770,9.770770,386.868132,2.640918,2.640918,51574.680220,5157.468022,19.389359,14.680371,58.077171,0.000000,56.280055,19.389359,13.220615,19.389359,123.232049,141.846372,0.00000,14.475378,44.729982,47.838808,0.000000,189.448962,51.205715,100000.000000,19.389359,0.000000,100000.000000,0.000000,2016,1,1.83,34.784141
1,1,2016-01-04,BANKNIFTY17000CE - I,183.9500,17000.0,0.1933,16590.1504,2016-01-28,BANKNIFTY17000PE - I,550.10,0.2012,AXISBANK450CE - I,9.90,450.0,0.3177,438.40,AXISBANK450PE - I,19.60,0.3082,NaN,0,0.00,0.0,0.0000,1.00,NaN,0,0,0.00,0.0000,BANKBARODA160CE - I,2.75,160.0,0.3781,151.20,BANKBARODA160PE - I,11.85,0.4380,2016-01-04,BANKINDIA117.5CE - I,2.90,117.5,0.3370,114.70,2016-01-28,BANKINDIA117.5PE - I,4.50,0.2658,2016-01-04,CANBK240CE - I,...,7.349070,0.000000,0.000000,0.000000,700.804396,0.750165,0.750165,1633.529670,2.316078,2.316078,279.817582,2.480652,2.480652,0.000000,0.000000,0.000000,2165.435165,9.811668,9.811668,374.350549,2.652147,2.652147,50399.936264,5039.993626,19.841295,14.680371,58.077171,0.000000,56.280055,19.389359,13.220615,19.389359,123.232049,141.846372,0.00000,14.475378,44.729982,47.838808,0.000000,189.448962,51.205715,100000.000000,19.389359,-198.781907,99801.218093,0.199176,2016,1,1.83,34.784141
2,2,2016-01-05,BANKNIFTY17000CE - I,156.4000,17000.0,0.1895,16545.1504,2016-01-28,BANKNIFTY17000PE - I,572.00,0.1993,AXISBANK450CE - I,8.55,450.0,0.3111,436.45,AXISBANK450PE - I,20.95,0.3178,NaN,0,0.00,0.0,0.0000,1.00,NaN,0,0,0.00,0.0000,BANKBARODA160CE - I,2.25,160.0,0.4159,148.10,BANKBARODA160PE - I,13.75,0.4254,2016-01-05,BANKINDIA117.5CE - I,2.35,117.5,0.3241,113.90,2016-01-28,BANKINDIA117.5PE - I,4.85,0.2562,2016-01-05

In [13]:
df1 = df[['Year', 'Month', 'Change_%_Monthly', 'Change_%_Yearly']]
df1 = df1.drop_duplicates()
df1

,Year,Month,Change_%_Monthly,Change_%_Yearly
0,2016,1,1.83,34.784141
20,2016,2,8.22,34.784141
41,2016,3,6.73,34.784141
61,2016,4,4.61,34.784141
79,2016,5,1.84,34.784141
101,2016,6,-0.01,34.784141
123,2016,7,5.44,34.784141
143,2016,8,1.00,34.784141
165,2016,9,-3.51,34.784141
185,2016,10,-2.96,34.784141


In [14]:
pivotTable = df1.pivot_table(values ='Change_%_Monthly', index =['Year', 'Change_%_Yearly'],
                         columns =['Month'])
pivotTable.columns = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
                      'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

pivotTable.index

MultiIndex([(2016,  34.78414145939301),
            (2017,  35.80075564764018),
            (2018, 21.964157898453784),
            (2019, 16.982329768144954),
            (2020,  55.14084143420557)],
           names=['Year', 'Change_%_Yearly'])

In [15]:
def _color_red_or_green(val):
    color = '#EE0000' if val < 0 else '#00EE00'
    return 'background-color: %s' % color
pivotTable.style.applymap(_color_red_or_green)


,,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
Year,Change_%_Yearly,,,,,,,,,,,,
2016,34.784141,1.830000,8.220000,6.730000,4.610000,1.840000,-0.010000,5.440000,1.000000,-3.510000,-2.960000,7.290000,0.550000
2017,35.800756,4.540000,7.270000,4.680000,2.790000,2.370000,1.530000,5.080000,3.350000,-1.620000,-5.560000,4.530000,2.650000
2018,21.964158,7.710000,4.990000,-0.100000,1.770000,0.720000,2.140000,3.480000,-9.010000,5.130000,-6.910000,9.220000,2.460000
2019,16.982330,1.030000,1.660000,3.730000,2.580000,-5.710000,-1.500000,3.950000,5.250000,5.230000,-4.600000,6.600000,-1.560000
2020,55.140841,5.880000,3.420000,12.930000,-2.370000,2.410000,1.970000,-1.620000,6.000000,6.920000,3.480000,2.760000,3.780000
